In [ ]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

import os
import re
from collections import defaultdict


def read_files(dir):
    return [f for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f))]

def sort_by_ext(file_dir: str,
                file_name: str):
    file_list = []
    file_list.append(file_name)
    ext_dict = defaultdict(list)
    for file in file_list:
        ext = file.split('.')[-1]
        file_path = os.path.join(file_dir, file)
        ext_dict[ext].append(file_path)
    return dict(ext_dict)

In [ ]:
def load_pdf(self, file_path):
    loader = PDFPlumberLoader(file_path)
    loaded_pdf = loader.load()
    for chunked in loaded_pdf:
        chunked.page_content = chunked.page_content.replace("\x00", "")
        for i in self.add_filter:
            chunked.page_content = re.sub(i, "#"*len(i), chunked.page_content)
        for k, v in self.def_metadata.items():
            chunked.metadata[str(k)] = v
        for k, v in self.add_metadata.items():
            chunked.metadata[str(k)] = v
    return loaded_pdf

def load_csv(self, file_path):
    loader = CSVLoader(file_path)
    loaded_csv = loader.load()
    for chunked in loaded_csv:
        chunked.page_content = chunked.page_content.replace("\x00", "")
        for i in self.add_filter:
            chunked.page_content = re.sub(i, "#"*len(i), chunked.page_content)
        for k, v in self.def_metadata.items():
            chunked.metadata[str(k)] = v
        for k, v in self.add_metadata.items():
            chunked.metadata[str(k)] = v
    return loaded_csv

def load_txt(self, file_path):
    loader = open(file_path)
    text = loader.read()
    for i in self.add_filter:
        text = re.sub(i, "#"*len(i), text)
    text = re.sub("\x00", "", text)
    metadata = {'source': file_path}
    for k, v in self.def_metadata.items():
        metadata[str(k)] = v
    for k, v in self.add_metadata.items():
        metadata[str(k)] = v
    loaded_txt = [Document(page_content=text, metadata=metadata)]
    return loaded_txt

def load_json(self, file_path):
    import pickle

    with open(file_path,"rb") as fr:
        loaded_json_rdb = pickle.load(fr)
    text = loaded_json_rdb['content']

    for i in self.add_filter:
        text = re.sub(i, "#"*len(i), text)
    text = re.sub("\x00", "", text)
    metadata = {'source': file_path}
    for k, v in self.def_metadata.items():
        metadata[str(k)] = v
    for k, v in self.add_metadata.items():
        metadata[str(k)] = v
    loaded_txt = [Document(page_content=text, metadata=metadata)]
    return loaded_txt

def load(self):
    result = {}
    files_by_ext = sort_by_ext(file_dir=self.file_dir, file_name=self.file_name)
    for k, v in files_by_ext.items():
        if k == 'pdf':
            for file_path in v:
                result[str(file_path.split('/')[-1])] = self.load_pdf(file_path)
        elif k == 'csv':
            for file_path in v:
                result[str(file_path.split('/')[-1])] = self.load_csv(file_path)
        elif k == 'txt':
            for file_path in v:
                result[str(file_path.split('/')[-1])] = self.load_txt(file_path)
        elif k == 'json':
            for file_path in v:
                result[str(file_path.split('/')[-1])] = self.load_json(file_path)
    return [result]